In [1]:
import requests
import jwt
import json
import base64
from requests.auth import HTTPBasicAuth

In [8]:
# Step 1: Obtain Access Token
def get_access_token(consumer_key, consumer_secret):
    token_url = "https://napi.kotaksecurities.com/oauth2/token"  # Replace with the actual token URL
    headers = {
        "Content-Type": "application/x-www-form-urlencoded"
    }
    payload = {
        "grant_type": "client_credentials"
    }
    response = requests.post(token_url, headers=headers, data=payload, auth=HTTPBasicAuth(consumer_key, consumer_secret))
    if response.status_code == 200:
        token_data = response.json()
        return token_data.get("access_token")
    else:
        print("Failed to get access token:", response.status_code, response.text)
        return None

In [15]:
# Step 2: Generate View Token
def get_view_token(access_token, mobile_number, password):
    url = "https://gw-napi.kotaksecurities.com/login/1.0/login/v2/validate"
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }
    body = {
        "mobileNumber": mobile_number,
        "password": password
    }
    response = requests.post(url, headers=headers, json=body)
    if response.status_code == 201:  # Assuming 201 is the correct status code for success
        response_data = response.json()["data"]  # Extracting data from the response
        view_token = response_data.get("token")
        sid = response_data.get("sid")
        return view_token, sid, response_data
    else:
        print("Failed to get view token and session ID:", response.status_code, response.text)
        return None, None, None

In [16]:
# Step 3: Correct padding for JWT and decode to get User ID
def decode_jwt(view_token):
    def add_padding(token):
        return token + '=' * (4 - len(token) % 4)
    
    segments = view_token.split('.')
    if len(segments) != 3:
        raise ValueError('Token structure incorrect')

    header_segment = add_padding(segments[0])
    payload_segment = add_padding(segments[1])
    
    try:
        header = json.loads(base64.urlsafe_b64decode(header_segment))
        payload = json.loads(base64.urlsafe_b64decode(payload_segment))
    except Exception as e:
        raise ValueError('Error decoding token: ' + str(e))
    
    user_id = payload.get("sub")
    return user_id

In [17]:
# Step 4: Generate OTP
def generate_otp(user_id):
    otp_url = "https://gw-napi.kotaksecurities.com/login/1.0/login/otp/generate"
    otp_body = {
        "userId": user_id,
        "sendEmail": True,
        "isWhitelisted": True
    }
    otp_response = requests.post(otp_url, json=otp_body)
    if otp_response.status_code == 200:
        print("OTP Generation Response:", otp_response.json())
        otp = input("Enter the OTP you received: ")
        return otp
    else:
        print("Failed to generate OTP:", otp_response.status_code, otp_response.text)
        return None

In [18]:
# Step 5: Generate Final Session Token
def generate_final_session_token(sid, token, access_token, user_id, otp):
    login_url = "https://gw-napi.kotaksecurities.com/login/1.0/login/v2/validate"
    headers = {
        "Sid": sid,
        "Auth": token,
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }
    login_body = {
        "userId": user_id,
        "otp": otp
    }
    login_response = requests.post(login_url, headers=headers, json=login_body)
    if login_response.status_code == 200:
        final_session_data = login_response.json()
        print("Final Session Token Response:", final_session_data)
    else:
        print("Failed to generate final session token:", login_response.status_code, login_response.text)

In [19]:
# Main Execution Flow
if __name__ == "__main__":
    # Inputs
    consumer_key = "MIML_AyI_Xadnb0MBJHCJ0VA7jca"
    consumer_secret = "HbNirWqfRNXOloew_TbTDdvOW0Ua"
    mobile_number = "+919818159195"  # Replace with your actual mobile number
    password = "A@1234b"  # Replace with your actual password
    
    # Step 1: Get Access Token
    access_token = get_access_token(consumer_key, consumer_secret)
    if access_token:
        # Step 2: Get View Token
        view_token, sid, response_data = get_view_token(access_token, mobile_number, password)
        if view_token and sid:
            # Step 3: Decode JWT to get User ID
            user_id = decode_jwt(view_token)
            print("User ID:", user_id)

            # Step 4: Generate OTP
            otp = generate_otp(user_id)
            if otp:
                # Step 5: Generate Final Session Token
                generate_final_session_token(sid, view_token, access_token, user_id, otp)

User ID: dd8508b3-d49f-4d32-8e17-c46d0d33f450
Failed to generate OTP: 401 {"code":"900902","message":"Missing Credentials","description":"Invalid Credentials. Make sure your API invocation call has a header: 'Authorization : Bearer ACCESS_TOKEN' or 'Authorization : Basic ACCESS_TOKEN' or 'apikey: API_KEY'"}
